# Early Stopping

If we are only interested in a model's intermediate computations, we can halt a forward pass run at any module level, reducing runtime and conserving compute resources. One examples where this could be particularly useful would if we are working with SAEs - we can train an SAE on one layer and then stop the execution. 

In [1]:
from nnsight import LanguageModel

model = LanguageModel('openai-community/gpt2', device_map='auto')

with model.trace("The Eiffel Tower is in the city of"):
   l1_out = model.transformer.h[0].output.save()
   model.transformer.h[0].output.stop()

# get the output of the first layer and stop tracing
print("L1 - Output: ", l1_out)

/opt/anaconda3/envs/nnsight/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


L1 - Output:  (tensor([[[ 0.1559, -0.7946,  0.3943,  ..., -1.7854, -0.4689,  0.1554],
         [-0.0624, -0.3135,  0.7256,  ..., -1.5666, -0.4111, -0.5625],
         [-0.3308, -0.3006, -0.9636,  ..., -1.3112, -0.5972,  0.3754],
         ...,
         [-1.3608,  0.6902,  0.2851,  ...,  0.1985,  0.1965, -0.5839],
         [-2.4975,  0.6122, -2.2314,  ...,  0.7751,  0.3189, -0.2851],
         [ 1.6383,  0.9466,  0.6050,  ..., -0.2360,  0.1267,  0.2722]]],
       device='mps:0', grad_fn=<AddBackward0>), (tensor([[[[-0.9420,  1.9023,  0.8722,  ..., -1.2703, -0.4792,  1.2469],
          [-1.9590,  2.7141,  2.8423,  ..., -1.1633, -1.6173,  2.1507],
          [-2.6123,  2.0937,  0.9679,  ..., -0.9763, -1.2243,  2.0279],
          ...,
          [-2.4282,  2.4462,  2.1550,  ..., -0.5916, -1.6641,  2.1119],
          [-3.5624,  3.6804,  2.5053,  ..., -0.3572, -2.5960,  0.9592],
          [-2.6021,  2.8035,  1.7291,  ..., -0.8557, -2.1589,  2.6881]],

         [[ 0.1103,  0.6967, -1.1409,  ..., -

Interventions within the tracing context do not necessarily execute in the order they are defined. Instead, their execution is tied to the module they are associated with.

As a result, if the forward pass is terminated early any interventions linked to modules beyond that point will be skipped, even if they were defined earlier in the context.

In the example below, the output of layer 2 _**cannot**_ be accessed since the model's execution was stopped at layer 1.

In [2]:
with model.trace("The Eiffel Tower is in the city of"):
   l2_out = model.transformer.h[1].output.save()
   model.transformer.h[0].output.stop()

print("L2 - Output: ", l2_out)

L2 - Output:  

ValueError: Accessing value before it's been set.